In [1]:
%load_ext lab_black

# Get a 'detector' object

In [2]:
# Get a `Detector` object
import shelve

with shelve.open("arctic_comparison", "r") as dct:
    detector = dct["detector"]

detector

In [3]:
import numpy as np
import typing as t

# Create densities and release timescales for 5 traps

In [4]:
rng = np.random.default_rng(seed=1234)

densities = rng.normal(loc=100.0, scale=5, size=5)
release_timescales = rng.normal(loc=1.2, scale=0.3, size=5)

In [5]:
densities

array([ 91.98081597, 100.32049957, 103.70445648, 100.76309597,
       104.31871946])

In [6]:
release_timescales

array([2.07392977, 0.75635299, 1.48364189, 0.70015936, 1.30312337])

# Test with Arctic Vanilla

First version using ``arctic`` library from PyPI

In [25]:
import arcticpy as ac


def arctic_00(
    detector,
    well_fill_power: float,
    density_1d: t.Sequence[float],
    release_timescale_1d: t.Sequence[float],
    express=0,
):

    char = detector.characteristics
    image = detector.pixel.array
    image = image.astype(float)
    ccd = ac.CCD(well_fill_power=well_fill_power, full_well_depth=char.fwc)
    roe = ac.ROE()

    assert len(density_1d) == len(release_timescale_1d)
    traps = []  # type: t.List[Trap]
    for density, release_timescale in zip(density_1d, release_timescale_1d):
        trap = ac.Trap(density=density, release_timescale=release_timescale)
        traps.append(trap)

    image_cti_added = ac.add_cti(
        image=image,
        parallel_traps=[trap],
        parallel_ccd=ccd,
        parallel_roe=roe,
        parallel_express=express,
    )

    image_cti_removed = ac.remove_cti(
        image=image_cti_added,
        iterations=1,
        parallel_traps=[trap],
        parallel_ccd=ccd,
        parallel_roe=roe,
        parallel_express=express,
    )

    detector.pixel.array = image_cti_removed

    return image_cti_removed

## Run for 1 trap

In [26]:
%%time

result_arctic_vanilla_1_trap = arctic_00(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=[100.0], 
    release_timescale_1d=[1.2], 
    express=0 
)

CPU times: user 34.7 s, sys: 103 ms, total: 34.8 s
Wall time: 35 s


In [9]:
%%timeit

# 12.9 s ± 572 ms per loop
_= arctic_00(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=[100.0], 
    release_timescale_1d=[1.2], 
    express=0 
)

KeyboardInterrupt: 

## Run for 5 traps

In [27]:
%%time

result_arctic_vanilla_5_traps = arctic_00(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=densities, 
    release_timescale_1d=release_timescales, 
    express=0 
)

CPU times: user 37.6 s, sys: 118 ms, total: 37.7 s
Wall time: 37.9 s


In [ ]:
%%timeit

# 16.7 s ± 247 ms per loop
_ = arctic_00(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=densities, 
    release_timescale_1d=release_timescales, 
    express=0 
)

# Arctic extracted without numba

First version with a modified `arctic` library

In [12]:
from arctic_without_numba import CCD as CCD_no_numba
from arctic_without_numba import ROE as ROE_no_numba
from arctic_without_numba import TrapsInstantCapture as TrapsInstantCapture_no_numba
from arctic_without_numba import add_cti as add_cti_no_numba
from arctic_without_numba import remove_cti as remove_cti_no_numba

In [28]:
def arctic_01(
    detector,
    well_fill_power: float,
    density_1d: t.Sequence[float],
    release_timescale_1d: t.Sequence[float],
    express=0,
):

    char = detector.characteristics
    image_2d = detector.pixel.array
    image_2d = image_2d.astype(float)

    ccd = CCD_no_numba(
        n_phases=1,
        fraction_of_traps_per_phase=np.array([1.0], dtype=np.float64),
        full_well_depth=np.array([char.fwc], dtype=np.float64),
        well_notch_depth=np.array([0.0], dtype=np.float64),
        well_fill_power=np.array([well_fill_power], dtype=np.float64),
        well_bloom_level=np.array([char.fwc], dtype=np.float64),
    )

    parallel_roe = ROE_no_numba(dwell_times=np.array([1.0], dtype=np.float64))
    # serial_roe = ROE(dwell_times=np.array([1.0], dtype=np.float64))
    # trap = Trap(density=density, release_timescale=release_timescale)

    # Create the trap(s)
    n_traps = len(density_1d)
    traps = TrapsInstantCapture_no_numba(
        density_1d=density_1d,
        release_timescale_1d=release_timescale_1d,
        surface_1d=np.array([False] * n_traps, dtype=np.bool_),
    )

    traps_lst = []
    traps_lst.append(traps)

    image_cti_added = add_cti_no_numba(
        image_2d=image_2d,
        parallel_traps=traps_lst,
        parallel_ccd=ccd,
        parallel_roe=parallel_roe,
        parallel_express=express,
        # serial_roe=serial_roe,
    )

    image_cti_removed = remove_cti_no_numba(
        image_2d=image_cti_added,
        iterations=1,
        parallel_traps=traps_lst,
        parallel_ccd=ccd,
        parallel_roe=parallel_roe,
        parallel_express=express,
        # serial_roe=serial_roe,
    )

    return image_cti_removed

## Run for 1 trap

In [29]:
%%time

result_arctic_no_numba_1_trap = arctic_01(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=np.array([100.0]), 
    release_timescale_1d=np.array([1.2]), 
    express=0 
)

/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_without_numba.py:948: RuntimeWarning: divide by zero encountered in true_divide
  self.capture_rate_1d = 1.0 / self.capture_timescale_1d  # type: np.ndarray
/System/Volumes/Data/work/sw/pyxel/pyxel/models/charge_transfer/arctic_without_numba.py:1164: RuntimeWarning: invalid value encountered in multiply
  fill_probabilities_from_empty_1d = self.capture_rates_1d * exponential_factor


CPU times: user 22.5 s, sys: 146 ms, total: 22.7 s
Wall time: 23.1 s


In [ ]:
%%timeit

# 8.11 s ± 132 ms per loop
_ = arctic_01(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=np.array([100.0]), 
    release_timescale_1d=np.array([1.2]), 
    express=0 
)

## Run for 5 traps

In [ ]:
%%time

result_arctic_no_numba_5_traps = arctic_01(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=densities, 
    release_timescale_1d=release_timescales, 
    express=0 
)

In [ ]:
%%timeit

# 11 s ± 262 ms per loop
_ = arctic_01(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=densities, 
    release_timescale_1d=release_timescales, 
    express=0 
)

## Comparison with vanilla arctic

In [ ]:
# Compare results from vanilla 'arctic' and this 'arctic'
np.testing.assert_equal(result_arctic_vanilla_1_trap, result_arctic_no_numba_1_trap)
np.testing.assert_equal(result_arctic_vanilla_5_traps, result_arctic_no_numba_5_traps)

# Arctic with numba


In [18]:
from arctic_with_numba_02 import CCD as CCD_numba
from arctic_with_numba_02 import ROE as ROE_numba
from arctic_with_numba_02 import TrapsInstantCapture as TrapsInstantCapture_numba
from arctic_with_numba_02 import add_cti as add_cti_numba
from arctic_with_numba_02 import remove_cti as remove_cti_numba

from numba.typed import List

In [21]:
def arctic_02(
    detector,
    well_fill_power: float,
    density_1d: t.Sequence[float],
    release_timescale_1d: t.Sequence[float],
    express=0,
):

    char = detector.characteristics
    image_2d = detector.pixel.array
    image_2d = image_2d.astype(float)

    ccd = CCD_numba(
        n_phases=1,
        fraction_of_traps_per_phase=np.array([1.0], dtype=np.float64),
        full_well_depth=np.array([char.fwc], dtype=np.float64),
        well_notch_depth=np.array([0.0], dtype=np.float64),
        well_fill_power=np.array([well_fill_power], dtype=np.float64),
        well_bloom_level=np.array([char.fwc], dtype=np.float64),
    )

    parallel_roe = ROE_numba(dwell_times=np.array([1.0], dtype=np.float64))
    # serial_roe = ROE(dwell_times=np.array([1.0], dtype=np.float64))
    # trap = Trap(density=density, release_timescale=release_timescale)

    # Create the trap(s)
    n_traps = len(density_1d)
    traps = TrapsInstantCapture_numba(
        density_1d=density_1d,
        release_timescale_1d=release_timescale_1d,
        surface_1d=np.array([False] * n_traps, dtype=np.bool_),
    )

    traps_lst = List()
    traps_lst.append(traps)

    image_cti_added = add_cti_numba(
        image_2d=image_2d,
        parallel_traps=traps_lst,
        parallel_ccd=ccd,
        parallel_roe=parallel_roe,
        parallel_express=express,
        # serial_roe=serial_roe,
    )

    image_cti_removed = remove_cti_numba(
        image_2d=image_cti_added,
        iterations=5,
        parallel_traps=traps_lst,
        parallel_ccd=ccd,
        parallel_roe=parallel_roe,
        parallel_express=express,
        # serial_roe=serial_roe,
    )
    return image_cti_removed

## Run for 1 trap

In [23]:
%%time

result_arctic_with_numba_1_trap = arctic_02(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=np.array([100.0], dtype=np.float64), 
    release_timescale_1d=np.array([1.2], dtype=np.float64), 
    express=0 
)

CPU times: user 6.55 s, sys: 43.7 ms, total: 6.59 s
Wall time: 6.56 s


In [ ]:
%%timeit

# 790 ms ± 35.3 ms per loop
_ = arctic_02(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=np.array([100.0]), 
    release_timescale_1d=np.array([1.2]), 
    express=0 
)

## Run for 5 traps

In [ ]:
%%time

result_arctic_with_numba_5_traps = arctic_02(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=densities, 
    release_timescale_1d=release_timescales, 
    express=0 
)

In [ ]:
%%timeit

# 1.85 s ± 59.1 ms per loop
_ = arctic_02(
    detector=detector,
    well_fill_power=0.8, 
    density_1d=densities, 
    release_timescale_1d=release_timescales, 
    express=0 
)

## Comparison with vanilla arctic

In [ ]:
# Compare results from vanilla 'arctic' and this 'arctic'
np.testing.assert_almost_equal(
    result_arctic_vanilla_1_trap, result_arctic_with_numba_1_trap
)
np.testing.assert_almost_equal(
    result_arctic_vanilla_5_traps, result_arctic_with_numba_5_traps
)

**Comparison with 1 Trap**


| Model           |  Time  | Speedup |
| --------------- |:------:| -------:| 
| Vanilla Arctic  | 12.9 s |  16.3x  |
| No Numba Arctic | 8.11 s |  10.3x  |
| Numba Arctic    | 790 ms |  1x     |


**Comparison with 5 Traps**


| Model           |  Time  | Speedup |
| --------------- |:------:| -------:| 
| Vanilla Arctic  | 16.7 s |  9x  |
| No Numba Arctic | 11.0 s |  5.9x  |
| Numba Arctic    | 1.86 s |  1x     |